# Ejercicio 02: Cálculo de la Matriz TF-IDF y Búsqueda de Consultas en un Corpus

El objetivo de este ejercicio es calcular la matriz TF-IDF de un corpus de documentos y luego aplicar una serie de consultas para recuperar los documentos más relevantes. Este ejercicio te ayudará a comprender cómo funciona el modelo de espacio vectorial y cómo se utiliza TF-IDF para ponderar términos en documentos y consultas.

In [17]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**1. Preprocesamiento del texto**

Lectura del corpus desde el archivo TXT.

In [18]:
# Abrir y leer el archivo que contiene el corpus desde la ruta específica
with open('/content/sample_data/02tfidfmatrix_corpus.txt', 'r', encoding='utf-8') as archivo:
    documentos = archivo.readlines()

# Mostrar el contenido del corpus para revisión
print("Texto cargado en el corpus:")
for indice, texto in enumerate(documentos):
    print(f"Texto {indice + 1}: {texto.strip()}")


Texto cargado en el corpus:
Texto 1: Documento 1: La inteligencia artificial continúa avanzando rápidamente, transformando sectores como la salud y las finanzas. Las empresas están adoptando algoritmos de aprendizaje automático para mejorar la eficiencia. Sin embargo, el desafío principal sigue siendo garantizar que las decisiones basadas en datos sean justas y no perpetúen sesgos. La ética es fundamental en este contexto.
Texto 2: Documento 2: El desarrollo de videojuegos ha alcanzado un nuevo nivel con tecnologías como la realidad aumentada y la inteligencia artificial. Los jugadores ahora pueden interactuar en mundos virtuales más inmersivos. Este crecimiento también impulsa el mercado de los e-sports, donde las competencias profesionales atraen a millones de espectadores en todo el mundo.
Texto 3: Documento 3: El comercio electrónico ha cambiado la forma en que compramos. Con la creciente demanda de compras en línea, las empresas están optimizando sus plataformas digitales para ofr

Tokenización de los documentos.

In [21]:
import re  # Asegurarse de que las expresiones regulares estén disponibles

def dividir_en_palabras(texto):
    # Utilizar expresiones regulares para segmentar el texto en términos
    palabras = re.findall(r'\b\w+\b', texto.lower())  # Convertir a minúsculas y capturar solo palabras
    return palabras

# Tokenizar cada documento dentro del conjunto de datos
documentos_tokenizados = [dividir_en_palabras(documento) for documento in documentos]

# Mostrar el resultado del corpus tokenizado
print("Documentos procesados:")
for indice, lista_palabras in enumerate(documentos_tokenizados):
    print(f"Texto {indice + 1}: {lista_palabras}")


Documentos procesados:
Texto 1: ['documento', '1', 'la', 'inteligencia', 'artificial', 'continúa', 'avanzando', 'rápidamente', 'transformando', 'sectores', 'como', 'la', 'salud', 'y', 'las', 'finanzas', 'las', 'empresas', 'están', 'adoptando', 'algoritmos', 'de', 'aprendizaje', 'automático', 'para', 'mejorar', 'la', 'eficiencia', 'sin', 'embargo', 'el', 'desafío', 'principal', 'sigue', 'siendo', 'garantizar', 'que', 'las', 'decisiones', 'basadas', 'en', 'datos', 'sean', 'justas', 'y', 'no', 'perpetúen', 'sesgos', 'la', 'ética', 'es', 'fundamental', 'en', 'este', 'contexto']
Texto 2: ['documento', '2', 'el', 'desarrollo', 'de', 'videojuegos', 'ha', 'alcanzado', 'un', 'nuevo', 'nivel', 'con', 'tecnologías', 'como', 'la', 'realidad', 'aumentada', 'y', 'la', 'inteligencia', 'artificial', 'los', 'jugadores', 'ahora', 'pueden', 'interactuar', 'en', 'mundos', 'virtuales', 'más', 'inmersivos', 'este', 'crecimiento', 'también', 'impulsa', 'el', 'mercado', 'de', 'los', 'e', 'sports', 'donde', 'l

Normalización del texto (conversión a minúsculas, eliminación de signos de puntuación).

In [24]:
def limpiar_texto(texto):
    # Convertir todo el texto a minúsculas
    texto = texto.lower()
    # Remover puntuaciones utilizando expresiones regulares
    texto = re.sub(r'[^\w\s]', '', texto)

    # Sustituir caracteres con acentos por sus equivalentes sin acentos
    # Crear una tabla de mapeo para la conversión
    mapa_traduccion = str.maketrans(
        "áéíóúüñ",  # Caracteres originales
        "aeiouun"   # Caracteres sustitutos
    )
    texto = texto.translate(mapa_traduccion)

    return texto

# Normalizar cada texto dentro del conjunto de documentos
documentos_normalizados = [limpiar_texto(documento) for documento in documentos]

# Mostrar el contenido normalizado
print("Textos normalizados:")
for indice, contenido in enumerate(documentos_normalizados):
    print(f"Texto {indice + 1}: {contenido}")


Textos normalizados:
Texto 1: documento 1 la inteligencia artificial continua avanzando rapidamente transformando sectores como la salud y las finanzas las empresas estan adoptando algoritmos de aprendizaje automatico para mejorar la eficiencia sin embargo el desafio principal sigue siendo garantizar que las decisiones basadas en datos sean justas y no perpetuen sesgos la etica es fundamental en este contexto

Texto 2: documento 2 el desarrollo de videojuegos ha alcanzado un nuevo nivel con tecnologias como la realidad aumentada y la inteligencia artificial los jugadores ahora pueden interactuar en mundos virtuales mas inmersivos este crecimiento tambien impulsa el mercado de los esports donde las competencias profesionales atraen a millones de espectadores en todo el mundo

Texto 3: documento 3 el comercio electronico ha cambiado la forma en que compramos con la creciente demanda de compras en linea las empresas estan optimizando sus plataformas digitales para ofrecer mejores experien

**2. Construcción de la matriz TF-IDF**

Cálculo de la frecuencia de término (TF) para cada término en cada documento.

In [25]:
# Configurar el vectorizador para calcular las frecuencias de los términos
frecuenciador = CountVectorizer()

# Generar la matriz de frecuencias a partir del corpus previamente normalizado
matriz_frecuencias = frecuenciador.fit_transform(documentos_normalizados)

# Convertir la matriz a un DataFrame para facilitar su interpretación
df_frecuencias = pd.DataFrame(matriz_frecuencias.toarray(), columns=frecuenciador.get_feature_names_out())

# Mostrar la matriz de frecuencias de términos (TF)
print("Matriz de Frecuencias de Términos (TF):")
print(df_frecuencias)


Matriz de Frecuencias de Términos (TF):
   acceder  accesibles  acceso  ademas  adopcion  adoptando  ahora  alcanzado  \
0        0           0       0       0         0          1      0          0   
1        0           0       0       0         0          0      1          1   
2        0           0       0       0         0          0      0          0   
3        1           0       0       0         0          0      0          0   
4        0           0       2       1         0          0      0          0   
5        0           0       0       0         0          0      0          0   
6        0           0       0       0         1          0      0          0   
7        0           1       0       0         0          0      0          0   

   algoritmos  anos  ...  una  uno  usuario  utilizan  vez  viaje  vida  \
0           1     0  ...    0    0        0         0    0      0     0   
1           0     0  ...    0    0        0         0    0      0     0   
2    

Cálculo de la frecuencia inversa de documento (IDF) para cada término en el corpus.

In [26]:
# Contar el total de textos en el conjunto de datos procesados
total_documentos = len(documentos_normalizados)

# Calcular la cantidad de textos que contienen cada término
frecuencia_documentos = (df_frecuencias > 0).sum()

# Calcular los valores IDF utilizando la fórmula matemática
valores_idf = np.log(total_documentos / frecuencia_documentos)

# Convertir los valores IDF en un DataFrame para mejor presentación
df_idf = pd.DataFrame(valores_idf, columns=['IDF']).reset_index()
df_idf.columns = ['Palabra', 'IDF']

# Mostrar los valores calculados de IDF
print("Frecuencia Inversa de Documento (IDF):")
print(df_idf)


Frecuencia Inversa de Documento (IDF):
         Palabra       IDF
0        acceder  2.079442
1     accesibles  2.079442
2         acceso  2.079442
3         ademas  2.079442
4       adopcion  2.079442
..           ...       ...
223        viaje  2.079442
224         vida  1.386294
225  videojuegos  2.079442
226    virtuales  2.079442
227     visuales  2.079442

[228 rows x 2 columns]


In [29]:
# Organizar los valores IDF en orden descendente
idf_valores_ordenados = df_idf.sort_values(by='IDF', ascending=False)

# Mostrar los valores IDF organizados de mayor a menor
print("Valores IDF en orden descendente:")
print(idf_valores_ordenados)

Valores IDF en orden descendente:
       Palabra       IDF
0      acceder  2.079442
134   medicina  2.079442
136  mejorando  2.079442
138    mejores  2.079442
139    mercado  2.079442
..         ...       ...
73          el  0.133531
126        las  0.133531
53          de  0.000000
66   documento  0.000000
125         la  0.000000

[228 rows x 2 columns]


In [30]:
# Transformar la matriz de frecuencias (TF) en un arreglo de NumPy para realizar operaciones
matriz_tf_numpy = df_frecuencias.values

# Generar la matriz TF-IDF al multiplicar TF por los valores IDF
matriz_tfidf = matriz_tf_numpy * valores_idf.values

# Convertir la matriz TF-IDF en un DataFrame para una mejor representación
df_tfidf = pd.DataFrame(matriz_tfidf, columns=df_idf['Palabra'])

# Mostrar la matriz TF-IDF
print("Matriz TF-IDF calculada:")
print(df_tfidf)


Matriz TF-IDF calculada:
Palabra   acceder  accesibles    acceso    ademas  adopcion  adoptando  \
0        0.000000    0.000000  0.000000  0.000000  0.000000   2.079442   
1        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
2        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
3        2.079442    0.000000  0.000000  0.000000  0.000000   0.000000   
4        0.000000    0.000000  4.158883  2.079442  0.000000   0.000000   
5        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
6        0.000000    0.000000  0.000000  0.000000  2.079442   0.000000   
7        0.000000    2.079442  0.000000  0.000000  0.000000   0.000000   

Palabra     ahora  alcanzado  algoritmos      anos  ...       una       uno  \
0        0.000000   0.000000    2.079442  0.000000  ...  0.000000  0.000000   
1        2.079442   2.079442    0.000000  0.000000  ...  0.000000  0.000000   
2        0.000000   0.000000    0.000000  0.000000  ...  0.000000  0.00

In [31]:
stopwords = [
    'la', 'las', 'el', 'los', 'de', 'y', 'en', 'a', 'que', 'con', 'por', 'para', 'como', 'más', 'es', 'se', 'esto', 'este',
    'esta', 'de', 'un', 'una', 'lo', 'al', 'del', 'su', 'las', 'ser', 'está', 'están', 'o', 'pero', 'no', 'si', 'con', 'su',
    'sobre', 'entre', 'este', 'también', 'esos', 'esas', 'esto', 'ese', 'por', 'ademas', 'cuando', 'ya', 'nos', 'tiene', 'nosotros',
    'mi', 'tú', 'te', 'su', 'y', 'se', 'un', 'al', 'desde', 'hasta', 'durante', 'aunque', 'al', 'muy', 'hay', 'donde', 'esto',
    'ser', 'una', 'los', 'mientras', 'cual', 'quien', 'que', 'sido', 'su', 'sobre', 'esto', 'ella', 'esto', 'yo', 'debería',
    'su', 'mi', 'tiempo', 'tal', 'vez', 'tan', 'uno', 'los', 'hace', 'muchos', 'otros', 'estas', 'ellas', 'nosotros', 'ellas',
    'algunas', 'algún', 'sólo', 'vez', 'que', 'más', 'del', 'menos', 'todo', 'al', 'siempre', 'por', 'también', 'en', 'todos'
]

In [32]:
# Seleccionar columnas de la matriz TF-IDF excluyendo las stopwords en español
columnas_relevantes = [palabra for palabra in df_tfidf.columns if palabra not in stopwords]

# Crear un nuevo DataFrame TF-IDF sin las stopwords
df_tfidf_filtrado = df_tfidf[columnas_relevantes]

# Mostrar la nueva matriz TF-IDF sin las stopwords
print("Matriz TF-IDF filtrada (sin Stopwords):")
print(df_tfidf_filtrado)


Matriz TF-IDF filtrada (sin Stopwords):
Palabra   acceder  accesibles    acceso  adopcion  adoptando     ahora  \
0        0.000000    0.000000  0.000000  0.000000   2.079442  0.000000   
1        0.000000    0.000000  0.000000  0.000000   0.000000  2.079442   
2        0.000000    0.000000  0.000000  0.000000   0.000000  0.000000   
3        2.079442    0.000000  0.000000  0.000000   0.000000  0.000000   
4        0.000000    0.000000  4.158883  0.000000   0.000000  0.000000   
5        0.000000    0.000000  0.000000  0.000000   0.000000  0.000000   
6        0.000000    0.000000  0.000000  2.079442   0.000000  0.000000   
7        0.000000    2.079442  0.000000  0.000000   0.000000  0.000000   

Palabra  alcanzado  algoritmos      anos  aplicaciones  ...  tratamientos  \
0         0.000000    2.079442  0.000000      0.000000  ...      0.000000   
1         2.079442    0.000000  0.000000      0.000000  ...      0.000000   
2         0.000000    0.000000  0.000000      0.000000  ...   

**3. Procesamiento de las consultas**

Preprocesamiento de las consultas de manera similar a los documentos.

In [33]:
# Lista de consultas para procesar
consultas = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

# Procesar cada consulta mediante la función de normalización
consultas_normalizadas = [limpiar_texto(consulta) for consulta in consultas]

# Mostrar las consultas normalizadas
print("Consultas procesadas:")
for indice, consulta in enumerate(consultas_normalizadas):
    print(f"Consulta {indice + 1}: {consulta}")


Consultas procesadas:
Consulta 1: inteligencia artificial en medicina
Consulta 2: beneficios de la educacion a distancia
Consulta 3: realidad aumentada en videojuegos
Consulta 4: desarrollo personal y habitos saludables
Consulta 5: futuro del comercio electronico
Consulta 6: tecnologias en cine moderno
Consulta 7: competencias de esports
Consulta 8: diagnostico con dispositivos portatiles
Consulta 9: literatura de ciencia ficcion
Consulta 10: plataformas de streaming


Representación de las consultas en el espacio vectorial TF-IDF.

In [34]:
# Procesar las consultas: normalización y tokenización
consultas_normalizadas = [limpiar_texto(consulta) for consulta in consultas]

# Tokenizar las consultas previamente normalizadas
consultas_tokenizadas = [dividir_en_palabras(consulta) for consulta in consultas_normalizadas]

# Filtrar las consultas para eliminar las stopwords
consultas_filtradas = []
for consulta in consultas_tokenizadas:
    consulta_limpia = [palabra for palabra in consulta if palabra not in stopwords]
    consultas_filtradas.append(consulta_limpia)

# Reconstruir las consultas sin stopwords como texto para pasarlas al vectorizador
consultas_preparadas = [' '.join(palabras) for palabras in consultas_filtradas]

# Configurar el vectorizador TF-IDF
vectorizador_tfidf = TfidfVectorizer()

# Generar la matriz TF-IDF y convertirla a un DataFrame
matriz_tfidf_consultas = vectorizador_tfidf.fit_transform(consultas_preparadas)
df_tfidf_consultas = pd.DataFrame(
    matriz_tfidf_consultas.toarray(),
    columns=vectorizador_tfidf.get_feature_names_out()
)

# Mostrar la matriz TF-IDF generada para las consultas
print("Matriz TF-IDF para las consultas procesadas:")
print(df_tfidf_consultas)


Matriz TF-IDF para las consultas procesadas:
   artificial  aumentada  beneficios  ciencia     cine  comercio  \
0     0.57735    0.00000     0.00000  0.00000  0.00000   0.00000   
1     0.00000    0.00000     0.57735  0.00000  0.00000   0.00000   
2     0.00000    0.57735     0.00000  0.00000  0.00000   0.00000   
3     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
4     0.00000    0.00000     0.00000  0.00000  0.00000   0.57735   
5     0.00000    0.00000     0.00000  0.00000  0.57735   0.00000   
6     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
7     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
8     0.00000    0.00000     0.00000  0.57735  0.00000   0.00000   
9     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   

   competencias  desarrollo  diagnostico  dispositivos  ...  medicina  \
0      0.000000         0.0      0.00000       0.00000  ...   0.57735   
1      0.000000         0.0      0.00000       0.00000  ... 

**4. Cálculo de similitudes**

Cálculo de la similitud entre cada consulta y los documentos del corpus utilizando la similitud del coseno.

In [37]:
# Configurar el vectorizador TF-IDF para procesar documentos y consultas
vectorizador_tfidf = TfidfVectorizer(stop_words=stopwords)  # Reutilizamos el mismo vectorizador para ambos

# Transformar el corpus de documentos en su representación TF-IDF
matriz_tfidf_documentos = vectorizador_tfidf.fit_transform(documentos)

# Aplicar la transformación TF-IDF a las consultas previamente preparadas
matriz_tfidf_consultas = vectorizador_tfidf.transform(consultas_preparadas)

# Calcular la similitud del coseno entre las consultas y los documentos
similitud_consultas_documentos = cosine_similarity(matriz_tfidf_consultas, matriz_tfidf_documentos)

# Convertir la matriz de similitudes en un DataFrame para facilitar la interpretación
df_similitud = pd.DataFrame(
    similitud_consultas_documentos,
    columns=[f'Texto {i+1}' for i in range(len(documentos))],
    index=[f'Pregunta {i+1}' for i in range(len(consultas))]
)

# Mostrar la matriz de similitud entre consultas y documentos
print("Matriz de Similitud del Coseno (Consultas vs Documentos):")
print(df_similitud)


Matriz de Similitud del Coseno (Consultas vs Documentos):
              Texto 1   Texto 2   Texto 3   Texto 4   Texto 5   Texto 6  \
Pregunta 1   0.131945  0.146161  0.000000  0.000000  0.000000  0.159074   
Pregunta 2   0.000000  0.000000  0.000000  0.272234  0.000000  0.000000   
Pregunta 3   0.000000  0.328565  0.000000  0.000000  0.073791  0.000000   
Pregunta 4   0.000000  0.085397  0.000000  0.000000  0.000000  0.000000   
Pregunta 5   0.000000  0.000000  0.148558  0.000000  0.000000  0.117100   
Pregunta 6   0.000000  0.000000  0.000000  0.000000  0.172705  0.000000   
Pregunta 7   0.000000  0.199871  0.000000  0.000000  0.000000  0.000000   
Pregunta 8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Pregunta 9   0.000000  0.000000  0.000000  0.000000  0.000000  0.217529   
Pregunta 10  0.000000  0.000000  0.104344  0.000000  0.225337  0.000000   

              Texto 7   Texto 8  
Pregunta 1   0.000000  0.278563  
Pregunta 2   0.000000  0.000000  
Pregunta 3   0

**5. Ranking de documentos**

Ordenar los documentos de mayor a menor similitud para cada consulta y mostrar los documentos más relevantes.

In [38]:
for indice_consulta, similitudes in df_similitud.iterrows():
    numero_consulta = int(re.search(r'\d+', indice_consulta).group())  # Extraer el número de la consulta

    texto_consulta = consultas[numero_consulta - 1]  # Recuperar el texto original de la consulta

    # Mostrar el ranking de documentos relacionados con cada consulta
    print(f"\nRanking de relevancia para la Pregunta {numero_consulta} - ({texto_consulta}):")

    # Ordenar los documentos según su similitud (de mayor a menor)
    ranking_relevancia = similitudes.sort_values(ascending=False)

    # Seleccionar los 3 documentos más relevantes
    top_relevantes = ranking_relevancia.head(3)
    for indice_documento, puntuacion in top_relevantes.items():
        # Mostrar el índice del documento y su puntuación de similitud
        print(f"**//** {indice_documento}   Similitud Coseno = {puntuacion:.6f} **//**")

        # Recuperar el texto del documento asociado
        texto_documento = documentos[int(indice_documento.split()[-1]) - 1].strip()
        print(f"{texto_documento}")



Ranking de relevancia para la Pregunta 1 - (inteligencia artificial en medicina):
**//** Texto 8   Similitud Coseno = 0.278563 **//**
Documento 8: Los avances en la medicina están cambiando la forma en que se diagnostican y tratan las enfermedades. Desde la telemedicina hasta los dispositivos portátiles que monitorean la salud en tiempo real, la tecnología está permitiendo a los médicos ofrecer cuidados más personalizados. El futuro de la medicina es prometedor, con tratamientos más accesibles y eficaces.
**//** Texto 6   Similitud Coseno = 0.159074 **//**
Documento 6: La ciencia ficción es uno de los géneros literarios más populares, explorando temas como el viaje en el tiempo, la inteligencia artificial y la vida en otros planetas. Este género no solo entretiene, sino que también invita a reflexionar sobre el futuro de la humanidad y las implicaciones de los avances tecnológicos.
**//** Texto 2   Similitud Coseno = 0.146161 **//**
Documento 2: El desarrollo de videojuegos ha alcanza